In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

In [2]:
# Image Size
img_size = 224

# Define the function to load and process images
def load_and_process_images(directory, classes, img_size):
    all_arrays = []
    for class_num, faceshape_class in enumerate(classes):
        path = os.path.join(directory, faceshape_class)
        print(f"Loading images from {path}")
        if not os.path.exists(path):
            print(f"Directory {path} does not exist.")
            continue
        for img_name in os.listdir(path):
            try:
                img_path = os.path.join(path, img_name)
                img_array = cv2.imread(img_path)
                if img_array is not None:
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                    img_array = cv2.resize(img_array, (img_size, img_size))
                    all_arrays.append([img_array, class_num])
                else:
                    print(f"Image {img_path} is None")
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")
    return np.array(all_arrays, dtype=object)

In [3]:

train_dir = "C:/Users/user/Downloads/Data settt/Boy/men/training_set"
test_dir = "C:/Users/user/Downloads/Data settt/Boy/men/testing_set"
classes = ["oval", "rectangular", "round", "square"]

In [4]:
# Load and process images
train_data = load_and_process_images(train_dir, classes, img_size)
test_data = load_and_process_images(test_dir, classes, img_size)

Loading images from C:/Users/user/Downloads/Data settt/Boy/men/training_set\oval
Directory C:/Users/user/Downloads/Data settt/Boy/men/training_set\oval does not exist.
Loading images from C:/Users/user/Downloads/Data settt/Boy/men/training_set\rectangular
Image C:/Users/user/Downloads/Data settt/Boy/men/training_set\rectangular\52µ.jpg is None
Loading images from C:/Users/user/Downloads/Data settt/Boy/men/training_set\round
Loading images from C:/Users/user/Downloads/Data settt/Boy/men/training_set\square
Loading images from C:/Users/user/Downloads/Data settt/Boy/men/testing_set\oval
Directory C:/Users/user/Downloads/Data settt/Boy/men/testing_set\oval does not exist.
Loading images from C:/Users/user/Downloads/Data settt/Boy/men/testing_set\rectangular
Loading images from C:/Users/user/Downloads/Data settt/Boy/men/testing_set\round
Loading images from C:/Users/user/Downloads/Data settt/Boy/men/testing_set\square


In [5]:
# Separate features and labels
x_train = np.array([i[0] for i in train_data])
y_train = to_categorical(np.array([i[1] for i in train_data]), num_classes=len(classes))
x_test = np.array([i[0] for i in test_data])
y_test = to_categorical(np.array([i[1] for i in test_data]), num_classes=len(classes))

In [6]:
# Initialize the model
base_model = VGG16(include_top=False, input_shape=(img_size, img_size, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=20, zoom_range=0.2, shear_range=0.2)
datagen.fit(x_train)

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=60), epochs=50, validation_data=(x_test, y_test))




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [7]:
# Save the model
model.save('boymodified.h5')

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Load the model and make a prediction
model = tf.keras.models.load_model('boymodified.h5')

# Define the image path and load the image
img_path = 'C:/Users/user/Downloads/Data settt/Boy/men/testing_set/rectangular/1.jpg'
img = image.load_img(img_path, target_size=(224, 224))

# Convert image to array
input_img = image.img_to_array(img)
input_img = np.expand_dims(input_img, axis=0)
input_img = preprocess_input(input_img)


In [9]:
# Predict the inputs on the model
predict_img = model.predict(input_img)

# Debugging: Print prediction probabilities and indices
print(f"Prediction probabilities: {predict_img}")

# Define your list of face shape classes
classes = ["oval", "rectangular", "round", "square"]

# Map predicted class indices to class labels
predicted_class_indices = np.argmax(predict_img, axis=1)
print(f"Predicted class indices: {predicted_class_indices}")

# Ensure predicted class indices are within range of classes list
predicted_classes = []
for idx in predicted_class_indices:
    if 0 <= idx < len(classes):
        predicted_classes.append(classes[idx])
    else:
        predicted_classes.append("Unknown")  # Handle out-of-range indices gracefully

# Print predicted face shape
print(f"Predicted face shape: {predicted_classes[0]}")

1/1 [==============================] - 1s 1s/step
Prediction probabilities: [[2.2607781e-05 1.8611681e-01 1.3848087e-01 6.7537969e-01]]
Predicted class indices: [3]
Predicted face shape: square


In [12]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model('boymodified.h5')

# Convert the model to a concrete function
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32)])
def model_func(input):
    return model(input)

concrete_func = model_func.get_concrete_function()

# Convert the Concrete Function to TFLite
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

# Save the TFLite model to disk
with open('boyvggmod12.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TFLite and saved as 'boyvgg.tflite'")


Model successfully converted to TFLite and saved as 'boyvgg.tflite'
